In [77]:
import copy
import os
import cobra
from tests import TEST_DIR
from diel_models.compartments_creator import CompartmentsCreator

ara_gem_model = os.path.join(TEST_DIR, "data", "AraGEM2010.xml")
non_diel_model = cobra.io.read_sbml_model(ara_gem_model)
non_diel_model_copy = copy.deepcopy(non_diel_model)

In [ ]:
diel_model = CompartmentsCreator(non_diel_model_copy)
diel_model.day_attribution()
DM = diel_model.duplicate()

In [55]:
import cobra

load_model = cobra.io.read_sbml_model("AraGEM2010_day_night_copy.xml")

In [56]:
from typing import List
import cobra
from cobra import Model, Metabolite, Reaction


class StoragePoolCreator:

    def __init__(self, model: Model, metabolites: List[str]):
        """
        Constructor

        Parameters
        ----------
        model: cobra.Model
        metabolites: List[str]
        """
        self.model: Model = model
        self.metabolites: List[str] = metabolites
        self.metabolite_sp_ids: List[str] = []
        self.metabolite_names: List[str] = []

    def create_storage_pool_metabolites(self) -> None:
        """
        According to the given metabolite IDs,
        this function creates new metabolites for a new compartment: storage pool

        Raises:
            ValueError: If a metabolite ID is not present in the given model.
        """
        metabolite_objs: List[Metabolite] = []
        names: List[str] = []

        for identification in self.metabolites:
            if identification in self.model.metabolites:
                metabolite_sp_id: str = identification.replace(self.model.metabolites.get_by_id(identification).compartment, "sp")
                names.append(self.model.metabolites.get_by_id(identification).name)
                self.metabolite_sp_ids.append(metabolite_sp_id)
            else:
                raise ValueError("Metabolite id not present in the model that was given.")
        for metabolite_sp_id in self.metabolite_sp_ids:
            metabolite = Metabolite(metabolite_sp_id)
            metabolite.compartment = "sp"
            metabolite_objs.append(metabolite)
        for idx, met in enumerate(metabolite_objs):
            met.name = names[idx]
            if "Day" in met.name:
                met.name = met.name.replace("Day", "")
            elif "Night" in met.name:
                met.name = met.name.replace("Night", "")
            self.metabolite_names.append(met.name)
        self.model.add_metabolites(metabolite_objs)
        self.model._compartments["sp"] = "Storage Pool"

    def create_exchange_reaction(self, metabolite_id: str, metabolite_sp_id: str, direction: str) -> Reaction:
        """
        It creates a Reaction object representing an exchange reaction
        for the specified metabolite in the specified direction.

        Parameters
        ----------
        metabolite_id
        metabolite_sp_id
        direction

        Returns
        -------
        Reaction object
        """

        met = self.model.metabolites.get_by_id(metabolite_sp_id)
        exchange_reaction = Reaction(f"{met.name}_{direction}_exchange".replace(" ", ""))
        if direction == "Day" and "Day" in metabolite_id:
            exchange_reaction.add_metabolites({
                self.model.metabolites.get_by_id(metabolite_id): -1.0,
                self.model.metabolites.get_by_id(metabolite_sp_id): 1.0
            })
        if direction == "Day" and "Night" in metabolite_id:
            exchange_reaction.add_metabolites({
                self.model.metabolites.get_by_id(metabolite_id.replace('Night', 'Day')): -1.0,
                self.model.metabolites.get_by_id(metabolite_sp_id): 1.0
            })
        if direction == "Night" and "Night" in metabolite_id:
            exchange_reaction.add_metabolites({
                self.model.metabolites.get_by_id(metabolite_sp_id): -1.0,
                self.model.metabolites.get_by_id(metabolite_id): 1.0
            })
        if direction == "Night" and "Day" in metabolite_id:
            exchange_reaction.add_metabolites({
                self.model.metabolites.get_by_id(metabolite_sp_id): -1.0,
                self.model.metabolites.get_by_id(metabolite_id.replace('Day', 'Night')): 1.0
            })
        exchange_reaction.lower_bound = -1000.0
        exchange_reaction.upper_bound = 1000.0
        exchange_reaction.name = f"{met.name} {direction} exchange"
        return exchange_reaction

    def create_storage_pool_first_reactions(self) -> None:
        """
        According to the given metabolite IDs,
        this function creates the corresponding exchange reaction
        of the metabolite to the storage pool.
        """
        exchange_reactions: List[Reaction] = []

        for metabolite_id, metabolite_sp_id in zip(self.metabolites, self.metabolite_sp_ids):
            if "Day" in metabolite_id:
                exchange_reactions.append(self.create_exchange_reaction(metabolite_id, metabolite_sp_id, "Day"))
            if "Night" in metabolite_id:
                exchange_reactions.append(self.create_exchange_reaction(metabolite_id, metabolite_sp_id, "Night"))

        self.model.add_reactions(exchange_reactions)

    def create_storage_pool_second_reactions(self) -> None:
        """
        According to the given metabolite IDs,
        this function creates the complementary reactions,
        i.e. if it previously created the Day <-> Storage Pool reaction,
        it now creates the Storage Pool <-> Night reaction and vice versa.
        """
        other_side_exchange_reactions: List[Reaction] = []

        for metabolite_id, metabolite_sp_id in zip(self.metabolites, self.metabolite_sp_ids):
            if "Day" in metabolite_id:
                other_side_exchange_reactions.append(self.create_exchange_reaction(metabolite_id, metabolite_sp_id, "Night"))
            if "Night" in metabolite_id:
                other_side_exchange_reactions.append(self.create_exchange_reaction(metabolite_id, metabolite_sp_id, "Day"))

        self.model.add_reactions(other_side_exchange_reactions)

In [57]:
test = StoragePoolCreator(load_model, ["S_Holo_45__91_carboxylase_93__c_Day","S_Homoeriodictyol_32_chalcone_c_Night", "S_Homogentisate_c_Day", "S_Hordenine_c_Day"])
test.create_storage_pool_metabolites()
test.create_storage_pool_first_reactions()

In [58]:
test.create_storage_pool_second_reactions()

In [59]:
load_model.reactions.query("exchange")

[<Reaction Holo-[carboxylase]_Day_exchange at 0x24868f75100>,
 <Reaction Homoeriodictyolchalcone_Night_exchange at 0x2486d2d9700>,
 <Reaction Homogentisate_Day_exchange at 0x2486d2d9580>,
 <Reaction Hordenine_Day_exchange at 0x2486d39dd00>,
 <Reaction Holo-[carboxylase]_Night_exchange at 0x2486a1da490>,
 <Reaction Homoeriodictyolchalcone_Day_exchange at 0x2486afed220>,
 <Reaction Homogentisate_Night_exchange at 0x24861d64c10>,
 <Reaction Hordenine_Night_exchange at 0x24869d8e130>]

In [60]:
load_model.reactions.get_by_id("Holo-[carboxylase]_Day_exchange")

Reaction identifier,Holo-[carboxylase]_Day_exchange
Name,Holo-[carboxylase] Day exchange
Memory address,0x24868f75100
Stoichiometry,S_Holo_45__91_carboxylase_93__c_Day <=> S_Holo_45__91_carboxylase_93__sp Holo-[carboxylase] Day <=> Holo-[carboxylase]
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [61]:
load_model.reactions.get_by_id("Holo-[carboxylase]_Night_exchange")

Reaction identifier,Holo-[carboxylase]_Night_exchange
Name,Holo-[carboxylase] Night exchange
Memory address,0x2486a1da490
Stoichiometry,S_Holo_45__91_carboxylase_93__sp <=> S_Holo_45__91_carboxylase_93__c_Night Holo-[carboxylase] <=> Holo-[carboxylase] Night
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [2]:
from typing import List
from cobra import Model, Metabolite, Reaction


class StoragePoolCreator:

    def __init__(self, model: Model, metabolites: List[str]):
        """
        Constructor

        Parameters
        ----------
        model: cobra.Model
        metabolites: List[str]
        """
        self.model: Model = model
        self.metabolites: List[str] = metabolites
        self.new_ids: List[str] = []
        self.met_names: List[str] = []

    def sp_metabolites(self) -> None:
        """
        According to the given metabolite IDs,
        this function creates new metabolites for a new compartment: storage pool

        Raises:
            ValueError: If a metabolite ID is not present in the given model.
        """
        metabolite_objs: List[Metabolite] = []
        names: List[str] = []

        for id in self.metabolites:
            if id in self.model.metabolites:
                new_id: str = id.replace(self.model.metabolites.get_by_id(id).compartment, "sp")
                names.append(self.model.metabolites.get_by_id(id).name)
                self.new_ids.append(new_id)
            else:
                raise ValueError("Metabolite id not present in the model that was given.")
        for new_id in self.new_ids:
            metabolite = Metabolite(new_id)
            metabolite.compartment = "sp"
            metabolite_objs.append(metabolite)
        for idx, met in enumerate(metabolite_objs):
            met.name = names[idx]
            if "Day" in met.name:
                met.name = met.name.replace("Day", "")
            elif "Night" in met.name:
                met.name = met.name.replace("Night", "")
            self.met_names.append(met.name)
        self.model.add_metabolites(metabolite_objs)
        self.model._compartments["sp"] = "Storage Pool"

    def sp_first_reactions(self) -> None:
        """
        According to the given metabolite IDs,
        this function creates the corresponding exchange reaction
        of the metabolite to the storage pool.
        """
        exchanges_r: List[Reaction] = []

        for met_id, new_id in zip(self.metabolites, self.new_ids):
            if "Day" in met_id:
                met = self.model.metabolites.get_by_id(met_id)
                exchange_r = Reaction(f"{met_id}_exchange")
                exchange_r.add_metabolites({
                    self.model.metabolites.get_by_id(met_id): -1.0,
                    self.model.metabolites.get_by_id(new_id): 1.0
                })
                exchange_r.lower_bound = -1000.0
                exchange_r.upper_bound = 1000.0
                exchange_r.name = f"{met.name} exchange"
                exchanges_r.append(exchange_r)
            if "Night" in met_id:
                met = self.model.metabolites.get_by_id(met_id)
                exchange_r = Reaction(f"{met_id}_exchange")
                exchange_r.add_metabolites({
                    self.model.metabolites.get_by_id(new_id): -1.0,
                    self.model.metabolites.get_by_id(met_id): 1.0
                })
                exchange_r.lower_bound = -1000.0
                exchange_r.upper_bound = 1000.0
                exchange_r.name = f"{met.name} exchange"
                exchanges_r.append(exchange_r)
        self.model.add_reactions(exchanges_r)

    def sp_second_reactions(self) -> None:
        """
        According to the given metabolite IDs,
        this function creates the complementary reactions,
        i.e. if it previously created the Day<->Storage Pool reaction,
        it now creates the Storage Pool <-> Night reaction and vice versa.
        """
        other_side_exchanges_r: List[Reaction] = []

        for met_id, new_id in zip(self.metabolites, self.new_ids):
            if "Day" in met_id:
                met = self.model.metabolites.get_by_id(met_id)
                other_side_exchange_r = Reaction(f"{met_id.replace('Day', 'Night')}_exchange")
                other_side_exchange_r.add_metabolites({
                    self.model.metabolites.get_by_id(new_id): -1.0,
                    self.model.metabolites.get_by_id(met_id.replace('Day', 'Night')): 1.0
                })
                other_side_exchange_r.lower_bound = -1000.0
                other_side_exchange_r.upper_bound = 1000.0
                other_side_exchange_r.name = f"{met.name.replace('Day', 'Night')} exchange"
                other_side_exchanges_r.append(other_side_exchange_r)
            if "Night" in met_id:
                met = self.model.metabolites.get_by_id(met_id)
                other_side_exchange_r = Reaction(f"{met_id.replace('Night', 'Day')}_exchange")
                other_side_exchange_r.add_metabolites({
                    self.model.metabolites.get_by_id(met_id.replace('Night', 'Day')): -1.0,
                    self.model.metabolites.get_by_id(new_id): 1.0
                })
                other_side_exchange_r.lower_bound = -1000.0
                other_side_exchange_r.upper_bound = 1000.0
                other_side_exchange_r.name = f"{met.name.replace('Night', 'Day')} exchange"
                other_side_exchanges_r.append(other_side_exchange_r)

        self.model.add_reactions(other_side_exchanges_r)

In [6]:
test = StoragePoolCreator(load_model, ["S_Holo_45__91_carboxylase_93__c_Day","S_Homoeriodictyol_32_chalcone_c_Night", "S_Homogentisate_c_Day", "S_Hordenine_c_Day"])
test.sp_metabolites()
test.sp_first_reactions()
test.sp_second_reactions()

In [7]:
load_model

Name,MODEL1507180028
Memory address,19269a20f40
Number of metabolites,3470
Number of reactions,3210
Number of genes,1404
Number of groups,0
Objective expression,0.01*Ex16_Day - 0.01*Ex16_Day_reverse_efba9
Compartments,"Cytosol Day, Plastid Day, Mitochondrion Day, Peroxisome Day, Vacuole Day, Biomass Day, Accumulation Day, External Day, Cytosol Night, Biomass Night, Plastid Night, Accumulation Night, External Night, Mitochondrion Night, Vacuole Night, Peroxisome Night, Storage Pool"


In [48]:
load_model.reactions.get_by_id("S_Homogentisate_c_Night_exchange")

Reaction identifier,S_Homogentisate_c_Night_exchange
Name,Homogentisate Night exchange
Memory address,0x23491b3e310
Stoichiometry,S_Homogentisate_sp <=> S_Homogentisate_c_Night Homogentisate <=> Homogentisate Night
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [49]:
load_model.reactions.get_by_id("S_Homogentisate_c_Day_exchange")

Reaction identifier,S_Homogentisate_c_Day_exchange
Name,Homogentisate Day exchange
Memory address,0x23486b26fa0
Stoichiometry,S_Homogentisate_c_Day <=> S_Homogentisate_sp Homogentisate Day <=> Homogentisate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [50]:
load_model.metabolites.query("_sp")

[<Metabolite S_Holo_45__91_carboxylase_93__sp at 0x23490a47be0>,
 <Metabolite S_Homoeriodictyol_32_chalcone_sp at 0x2349127baf0>,
 <Metabolite S_Homogentisate_sp at 0x2349127bac0>,
 <Metabolite S_Hordenine_sp at 0x2348c6041c0>]

In [51]:
load_model.compartments

{'c_Day': 'Cytosol Day',
 'p_Day': 'Plastid Day',
 'm_Day': 'Mitochondrion Day',
 'x_Day': 'Peroxisome Day',
 'v_Day': 'Vacuole Day',
 'biomass_Day': 'Biomass Day',
 'acc_Day': 'Accumulation Day',
 'ext_Day': 'External Day',
 'c_Night': 'Cytosol Night',
 'biomass_Night': 'Biomass Night',
 'p_Night': 'Plastid Night',
 'acc_Night': 'Accumulation Night',
 'ext_Night': 'External Night',
 'm_Night': 'Mitochondrion Night',
 'v_Night': 'Vacuole Night',
 'x_Night': 'Peroxisome Night',
 'sp': 'Storage Pool'}